## Descrição do Notebook

Neste notebook, os dados foram agregados por semana e as regras aplicadas estão descritas ao decorrer do código.

In [4]:
import pandas as pd 
import numpy as np
import xlrd
import matplotlib.pyplot as plt


### Lendo o arquivo e informando que a tabela DATA_TRAT. é do tipo datetime

In [75]:
df = pd.read_csv("../datasets/df_rpo_v3.csv")

df['DATA_TRAT.'] = pd.to_datetime(df["DATA_TRAT."], dayfirst = True)


### Atribuindo apenas dados referentes a 2017 à variável df

In [76]:
df = df.loc[(df['DATA_TRAT.'] >= '2017-01-01') & (df['DATA_TRAT.'] <= '2017-12-31')]

Retirando P10 e P90 das vendas dos produtos:

Criando o *Dataframe* agregado por semana

In [78]:
# Criando colunas de Semana e Ano para auxiliar na agregação
df.reset_index(inplace = True, drop = True)
df["Semana"] = df['DATA_TRAT.'].dt.isocalendar().week
df["Ano"] = df["DATA_TRAT."].dt.year
#df["Mes"] = df['DATA_TRAT.'].dt.month

> Fazendo as agregações: 
* Colunas de **Vendas Reais, Margem e CMV** foram somadas
* Colunas de **Preço  e Porcentagem de Margem** foram calculadas com base na média
* Coluna da razão **CMV/QTD** foi retirada uma nova fração entre a coluna CMV e QTD para respeitar a razão entre as grandezas.

### A coluna mês foi removida do agrupamento por estar contribuindo para a duplicação dos dados
- 17-01-2023

In [80]:
### Definindo df_Agg
df_agg = df[["SAP+NOME", "UF", "Ano","Semana",  "preço2", "Mg %",
                     "Margem Líquida Real %", "Preço"]]
### Definindo df_Agg_sum
df_agg_sum = df[["SAP+NOME", "UF", "Semana", "Vendas Reais Qtde", "Vendas Reais $",
                     "Margem Líquida Real $", "Venda Líquida Real $", "CMV"]]

In [81]:
### Agrupando df_agg por SAP+NOME, UF e Semana e calculando a média
df_agg = df_agg.groupby(["SAP+NOME", "UF", "Semana"]).mean().reset_index()

### Agrupando df_Agg_sum por SAP+NOME, UF e Semana e somando as demais variáveis
df_agg_sum = df_agg_sum.groupby(["SAP+NOME", "UF", "Semana"]).sum().reset_index()

### Calculando CMV individual
df_agg_sum["CMV/QTD"] = df_agg_sum["CMV"] / df_agg_sum["Vendas Reais Qtde"]

Unindo os dois *dataframes* de agregação criados para chegar ao *dataset* que será depositado no *Data Lake*

In [82]:
### Criando df_final como cópia do df_Agg
df_final = df_agg.copy()

### Fazendo merge entre df_final e df_agg_sum utilizando SAP+NOME, UF e Semana como Chaves
df_final = df_final.merge(df_agg_sum, on = ["SAP+NOME", "UF", "Semana"], how = "inner")

### Resetando index
df_final.reset_index(inplace = True, drop = True)

,SAP+NOME,UF,Semana,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,Vendas Reais $,Margem Líquida Real $,Venda Líquida Real $,CMV,CMV/QTD
0,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,2,2017.0,4.99,0.516304,51.630435,4.990000,2.0,9.98,3.80,7.36,3.56,1.780000
1,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,3,2017.0,4.99,0.508152,50.815217,4.990000,3.0,14.97,5.61,11.04,5.43,1.810000
2,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,6,2017.0,2.99,0.177273,17.727273,2.990000,1.0,2.99,0.39,2.20,1.81,1.810000
3,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,9,2017.0,4.99,0.508152,50.815217,4.990000,1.0,4.99,1.87,3.68,1.81,1.810000
4,000000000002158151 - T VISITA P BORDA DESIRE A...,AC,10,2017.0,4.99,0.508152,50.815217,4.990000,5.0,24.95,9.35,18.40,9.05,1.810000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11647,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,48,2017.0,4.99,0.325059,32.505907,4.940000,23.0,114.27,27.41,83.33,55.92,2.431304
11648,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,49,2017.0,4.99,0.274501,27.450107,4.617266,158.0,725.63,144.48,528.74,384.26,2.432025
11649,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,50,2017.0,4.99,0.300096,30.009580,4.765641,60.0,285.40,62.16,208.02,145.86,2.431000
11650,000000000004270164 - DESINF LIQ PINHO SOL ORIG...,SP,51,2017.0,4.99,0.302522,30.252185,4.793163,129.0,614.72,134.13,448.00,313.87,2.433101


### Verificação de dados duplicados

In [84]:
df_final.duplicated(subset=['SAP+NOME', 'UF', 'Semana']).value_counts()

False    11652
dtype: int64

Retirando *P10* e *P90* dos produtos que possuiram variações de preço mais de 10 vezes. Essa agregação foi feita por **Produto** e por **UF** para eliminar os preços promocionais e as sazonalidades de vendas causadas por eles. E para as demais situações de um dado **Produto** ou **UF**, onde o preço não variou mais de 10 vezes, mas, ainda sim, há sazonalidade na quantidade de vendas semanais, também foram removidos os valores menores que *P10*, para remover quantidades insignificantes e *P90*, para remover valores que destoam.

In [85]:
lst_uf = df_final["UF"].unique()
lst_sap = df_final["SAP+NOME"].unique()

for uf in lst_uf:
    for sap in lst_sap:
        df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
        if df_seg.empty == True:
               continue
        else:
            # Se preço variar mais de 10 vezes, encontraremos o p10 e o p90 de valor e quantidade
            if len(df_seg["Preço"].unique()) > 10:
                q10_p = np.quantile(df_seg["Preço"].unique(), .10)
                q90_p = np.quantile(df_seg["Preço"].unique(), .90)
                q_10_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .10)
                q_90_qtd = np.quantile(df_seg["Vendas Reais Qtde"], .90)
                
                # Atribuindo a 'ind', todos os valores que atendem as condições
                ind = df_seg.loc[((df_seg["Preço"] < q10_p)  | (df_seg["Preço"] > q90_p)) &
                                 ((df_seg["Vendas Reais Qtde"] < q_10_qtd) | (df_seg["Vendas Reais Qtde"] > q_90_qtd))]
                
                # Dropando index dos valores de 'ind'
                df_final = df_final.drop(ind.index)
            
            # Atribuindo a df_seg o valor de df_final ao final da condição
            df_seg = df_final.loc[(df_final["UF"] == uf) & (df_final["SAP+NOME"] == sap)].copy()
            
            # Pegando o p10 e o p90 da coluna Vendas Reais Qtde
            p10 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .10)
            p90 = np.quantile(df_seg["Vendas Reais Qtde"].unique(), .90)
            
            # Atribuindo a variável ind todos os index que são menores que o p10 e maiores que o p90
            ind = df_seg.loc[(df_seg["Vendas Reais Qtde"] < p10)  | (df_seg["Vendas Reais Qtde"] > p90)]
            
            # Removendo do DF todos os index que foram atribuidos a variável ind
            df_final = df_final.drop(ind.index)

### Ordenando o df final por Ano, Semana e UF

In [86]:
df_final = df_final.sort_values(by = ["Ano", "Semana", "UF"]).reset_index(drop = True)

### Gravando df_final como arquivo CSV

In [88]:
df_final.to_csv("../datasets/df_rpo_v4_2.csv", index = False)